In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from art.attacks.evasion import DeepFool
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np
import pandas as pd

from sklearn import metrics

import joblib

2024-02-26 06:00:24.459604: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 06:00:24.462404: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 06:00:24.494732: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 06:00:24.494779: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 06:00:24.496436: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [3]:
x_train = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/x_train.csv', low_memory=False)
y_train = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/y_train.csv', low_memory=False)
x_val = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/x_val.csv', low_memory=False)
y_val = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/y_val.csv', low_memory=False)
x_test = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/x_test.csv', low_memory=False)
y_test = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/y_test.csv', low_memory=False)

In [4]:
from sklearn import preprocessing
s1 = preprocessing.MinMaxScaler()
s2 = preprocessing.MinMaxScaler()
s3 = preprocessing.MinMaxScaler()

x_train_scaled = s1.fit_transform(x_train)
x_val_scaled = s2.fit_transform(x_val)
x_test_scaled = s3.fit_transform(x_test)

x_train = np.copy(x_train_scaled)
x_val = np.copy(x_val_scaled)
x_test = np.copy(x_test_scaled)

In [5]:
model = joblib.load("/home/jovyan/WUSTL-IIoT/dl/dl_new/dnn.joblib")

In [6]:
x_test_trimmed = x_test[31000:32600]
y_test_trimmed = y_test[31000:32600]

In [7]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=5,
    input_shape=(48,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [8]:
attack = DeepFool(classifier=classifier, epsilon=0.01)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool:   0%|          | 0/1600 [00:00<?, ?it/s]

DeepFool: 100%|██████████| 1600/1600 [49:25<00:00,  1.85s/it]


In [9]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

50/50 [==============================] - 0s 793us/step
Accuracy: 0.953125
AUC: 0.9282552518714061

macro
Precision: 0.5373208305425318
Recall: 0.6406593406593407
F1 Score: 0.4962954926087978

weighted
Precision: 0.9824799480910918
Recall: 0.953125
F1 Score: 0.9565747257557244

Mean FNR: 0.3593406593406593
Mean TNR: 0.9616142339680488
Mean FPR: 0.03838576603195121
Mean TPR: 0.6406593406593407


In [13]:
attack = DeepFool(classifier=classifier, epsilon=0.05)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool: 100%|██████████| 1600/1600 [49:46<00:00,  1.87s/it]


In [14]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

50/50 [==============================] - 0s 654us/step
Accuracy: 0.953125
AUC: 0.9271134384576365

macro
Precision: 0.5373208305425318
Recall: 0.6406593406593407
F1 Score: 0.4962954926087978

weighted
Precision: 0.9824799480910918
Recall: 0.953125
F1 Score: 0.9565747257557244

Mean FNR: 0.3593406593406593
Mean TNR: 0.9616142339680488
Mean FPR: 0.03838576603195121
Mean TPR: 0.6406593406593407


In [81]:
attack = DeepFool(classifier=classifier, epsilon=0.1)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool:   0%|          | 0/1600 [00:00<?, ?it/s]

DeepFool: 100%|██████████| 1600/1600 [1:10:33<00:00,  2.65s/it]


In [82]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

50/50 [==============================] - 0s 622us/step
Accuracy: 0.953125
AUC: 0.9250984779804663

macro
Precision: 0.5373208305425318
Recall: 0.6406593406593407
F1 Score: 0.4962954926087978

weighted
Precision: 0.9824799480910918
Recall: 0.953125
F1 Score: 0.9565747257557244

Mean FNR: 0.3593406593406593
Mean TNR: 0.9616142339680488
Mean FPR: 0.03838576603195121
Mean TPR: 0.6406593406593407


In [83]:
attack = DeepFool(classifier=classifier, epsilon=0.15)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool: 100%|██████████| 1600/1600 [1:10:53<00:00,  2.66s/it]


In [84]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

50/50 [==============================] - 0s 610us/step
Accuracy: 0.953125
AUC: 0.9245957651314217

macro
Precision: 0.5473208305425319
Recall: 0.6406593406593407
F1 Score: 0.5096288259421311

weighted
Precision: 0.9825111980910919
Recall: 0.953125
F1 Score: 0.956616392422391

Mean FNR: 0.3593406593406593
Mean TNR: 0.9616142339680488
Mean FPR: 0.03838576603195121
Mean TPR: 0.6406593406593407


In [85]:
attack = DeepFool(classifier=classifier, epsilon=0.2)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool: 100%|██████████| 1600/1600 [1:11:11<00:00,  2.67s/it]


In [86]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

50/50 [==============================] - 0s 579us/step
Accuracy: 0.953125
AUC: 0.9240942523591821

macro
Precision: 0.5973208305425318
Recall: 0.6406593406593407
F1 Score: 0.5629621592754643

weighted
Precision: 0.9826674480910919
Recall: 0.953125
F1 Score: 0.9567830590890577

Mean FNR: 0.3593406593406593
Mean TNR: 0.9616142339680488
Mean FPR: 0.03838576603195121
Mean TPR: 0.6406593406593407


In [87]:
attack = DeepFool(classifier=classifier, epsilon=0.25)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool: 100%|██████████| 1600/1600 [1:11:04<00:00,  2.67s/it]


In [88]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

50/50 [==============================] - 0s 572us/step
Accuracy: 0.953125
AUC: 0.8293662748071176

macro
Precision: 0.5975855130784709
Recall: 0.6406593406593407
F1 Score: 0.5630962020435705

weighted
Precision: 0.9838858148893361
Recall: 0.953125
F1 Score: 0.9574000747059956

Mean FNR: 0.3593406593406593
Mean TNR: 0.9645136839195441
Mean FPR: 0.03548631608045593
Mean TPR: 0.6406593406593407


In [89]:
attack = DeepFool(classifier=classifier, epsilon=0.3)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool: 100%|██████████| 1600/1600 [1:11:11<00:00,  2.67s/it]


In [90]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

50/50 [==============================] - 0s 591us/step
Accuracy: 0.953125
AUC: 0.8294324608920324

macro
Precision: 0.5975855130784709
Recall: 0.6406593406593407
F1 Score: 0.5630962020435705

weighted
Precision: 0.9838858148893361
Recall: 0.953125
F1 Score: 0.9574000747059956

Mean FNR: 0.3593406593406593
Mean TNR: 0.9645136839195441
Mean FPR: 0.03548631608045593
Mean TPR: 0.6406593406593407
